# Grid Reference Triangles

The request is for, *"An application that, by inputting three grid references (making a triangle), can work out and display distance and bearing, as well as internal angles."* 

Following clarification:

- Grids will be Ordnance Survey National Grid (Format SU625323)
- Distances are short so assuming: No magnetic variation and flat earth are fine
- Grids must contain opening 2 letter and an even number of digits but can be from 2 to 10 digit grids (10km to 1m)

There is a good summary of the National Grid system here: https://www.ordnancesurvey.co.uk/documents/resources/guide-to-nationalgrid.pdf

Attribution https://www.movable-type.co.uk/scripts/os-grid-dist.html Chris Veness from JS version

Math required for calcualtions with pi

In [ ]:
import math as m


## Setup Functions
Function to convert National Grid Reference to fully numeric 1m grid reference with 100km numeric grids (SU625323 -> 462500 132300)

In [ ]:
def gridref_numeric(gridref):
    # Check gridref has 2 letters at start and even amount of digits following
    if gridref.strip()[:2].isalpha() and gridref.strip()[2:].isnumeric() and len(gridref.strip()[2:])%2 == 0:
        # Convert letters to numbers A,B,C -> 0,1,2
        letter_E = ord(gridref.upper()[0]) - ord('A'.upper())
        letter_N = ord(gridref.upper()[1]) - ord('A'.upper())
        # Letter I is not used so adjust H,J,K -> 7,8,9
        if letter_E > 7:
            letter_E -= 1
        if letter_N > 7:
            letter_N -= 1
        # Convert grid letters into 100km-square index from false origin at grid SV (SU -> 4,1)
        easting = (((letter_E+3)%5)*5) + (letter_N%5)
        northing = (19-((m.floor(letter_E/5)))*5) - (m.floor(letter_N/5))
        # Get numerical part of gridref and split easting and northing (625323 -> 625, 323)
        gridref_numbers = gridref[2:]
        gr_easting = gridref_numbers[:len(gridref_numbers)//2]
        gr_northing = gridref_numbers[len(gridref_numbers)//2:]
        # Normalise to 1m grid / 10-figure grid reference (625 323 -> 62500 32300)
        gr_easting = '{:0<5}'.format(gr_easting)
        gr_northing = '{:0<5}'.format(gr_northing)
        # Add grid letter number to front and convert str to int (4 62500, 1 32300 -> 462500 132300)
        easting = int(str(easting) + gr_easting)
        northing = int(str(northing) + gr_northing)
        return easting,northing
    else:
        return print('Grid Reference is in wrong format')

def grid_distance(grid_1, grid_2):
    # Function to calculate distance between 2 National Grid References - can be different types but must be valid (SU625323 and NU3256)

    # Get fully numeric grids
    gr_1 = gridref_numeric(grid_1)
    gr_2 = gridref_numeric(grid_2)

    # Get easting and northing distances between grid_1 and grid_2
    delta_E = gr_2[0] - gr_1[0]
    delta_N = gr_2[1] - gr_1[1]

    # Use pythagoras to get distance between points (a^2 = b^2 + c^2)
    dist = round(m.sqrt(delta_E**2 + delta_N**2))
    return dist

def grid_bearing(grid_1, grid_2):
    # Function to calculate bearing between 2 National Grid References - can be different types but must be valid (SU625323 and NU3256)

    # Get fully numeric grids
    gr_1 = gridref_numeric(grid_1)
    gr_2 = gridref_numeric(grid_2)

    # Get easting and northing distances between grid_1 and grid_2
    delta_E = gr_2[0] - gr_1[0]
    delta_N = gr_2[1] - gr_1[1]

    # Use arctan to get the bearing convert from radians to degrees
    bearing = round((90-(m.atan2(delta_N, delta_E)/m.pi*180)+360) % 360)
    return bearing

def grid_angle(dist_a,dist_b,dist_c):
    # Function to find triangle angles between grid points (Point A has angle A. Distance a = B to C, b = A to C, c = A to B )
    angle_A = round(m.degrees(m.acos((dist_b**2 + dist_c**2 - dist_a**2) / (2 * dist_b * dist_c))),2)
    return angle_A


## Input Grids
Grids must start with the OS National Grid 2-letter prefix and contain an even number of digits up to a 10-figure grid (1m) (eg: SU625323, SU6232, SU6254032320)

In [ ]:
# Remove the dummy data and replace with your grids BELOW

grid_inputs = ['ST560230',      # Enter Grid A here
                'SU300200',     # Enter Grid B here
                'ST340040'      # Enter Grid C here
                ] 

In [ ]:
# Create pairs of grids: AB, BC, CA
grid_pairs = list(zip(grid_inputs, grid_inputs[1:] + grid_inputs[:1]))
# Distance between grids A - B called 'c' (BC - a, CA - b)  
dist_keys = ['a_BC','b_CA','c_AB']
dist_values = []

# Calculagte the length of each side of the triangle
for x, value in enumerate(grid_pairs):
    dist_val = grid_distance(*value)
    dist_values.append(dist_val)

# Arrange distances as a, b, c.
dist_values.append(dist_values.pop(0))
distances = dict(zip(dist_keys, dist_values))

angle_label = ['A','B','C']
angles = dict.fromkeys(angle_label, [])

for key in angles:
    # Calculate internal angle of triangle at grid
    angle = grid_angle(*dist_values)
    angles.update({key : angle})
    # Label the grid triangle corner
    to_grid1 = chr(ord(key)+1) if ord(key) < 67 else 'A'
    to_grid2 = chr(ord(key)-1) if ord(key) > 65 else 'C'
    # Calculate the bearing to the other 2 grids with correct label
    bearing1 = grid_bearing(*(grid_inputs[:2]))
    bearing2 = grid_bearing(grid_inputs[0],grid_inputs[2])
    # Output the 
    print('Grid',key,':',grid_inputs[0],'Angle', key,': ', angle , 'degrees / ', round(angle*(m.pi/0.180)) ,'milliradian /', round(angle*(160/9)), 'mils (NATO)')
    # Need to add bearings and distance for each point - below showing the distances but not bearings 
    print('From:', key, 'to', to_grid1, str(bearing1).zfill(3), 'degrees / ', round(bearing1*(m.pi/0.180)) ,'milliradian /', round(bearing1*(160/9)), 'mils (NATO)', 'at:', dist_values[1], 'metres')
    print('From:', key, 'to', to_grid2, str(bearing2).zfill(3), 'degrees / ', round(bearing2*(m.pi/0.180)) ,'milliradian /', round(bearing2*(160/9)), 'mils (NATO)', 'at:', dist_values[2], 'metres','\n')
    
    dist_values.append(dist_values.pop(0))
    grid_inputs.append(grid_inputs.pop(0))
